In [1]:
import pandas as pd
import os
import glob
import requests
from functools import reduce
from typing import Iterable
import math
import tqdm
from evaltools.data import census
import us
import json
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")

# In this first section of the notebook, I'm using census from evaltools to get VAP broken down by race for every block in Texas.

1) I make the call to census using the P4 table which gets NH racial categories + HVAP
2) I'm creating a mapping for each racial category in Texas to the list of columns in the returned dataframe. I ultimately only include (NH) BLACK, (NH) WHITE, HISP, (NH) ASIAN, (NH) OTHER
3) I sum all of these columns to create find the probability that someone in a given block is a certain race. 
    * Here I used 2020 data
    * There are also other combinations that can be made by calling census on multiple tables and combining. For example, calling P3 and P4 would give BLACK and (NH) BLACK, and by taking the difference we could find (H) Black. I have chosen not to do this, because the surname data doesn't have these breakdowns. 
4) I'm filling all nan-values with 0. 
5) Ultimately, I ran this and saved to a DataFrame, so this section doesn't have to be run each time 

** Mostly including to cover any methodology questions that may come up later. Note item number 5**

In [2]:
texas = us.states.TX
texas_vap_p4 = census(texas, table = "P4")

KeyboardInterrupt: 

This folder where this notebook is has the Texas_VAP_P4 csv, so the cell below can be changed to read in.

In [ ]:
texas_vap_p4.to_csv("Texas_VAP_P4.csv", index=False)

In [ ]:
texas_vap_p4 = pd.read_csv("Texas_VAP_P4.csv")

In [ ]:
texas_vap_p4.columns

In [ ]:
all_vap_cols = [col for col in texas_vap_p4.columns]
black_cols = [col for col in all_vap_cols if "BLACK" in col]
all_vap_cols = list(set(all_vap_cols) - set(black_cols))
asian_cols = [col for col in all_vap_cols if "ASIAN" in col]
all_vap_cols = list(set(all_vap_cols) - set(asian_cols))
hisp_cols = [col for col in all_vap_cols if col == "HVAP20"]
all_vap_cols = list(set(all_vap_cols) - set(hisp_cols))
white_cols = [col for col in all_vap_cols if col == "NHWHITEVAP20"]
all_vap_cols = list(set(all_vap_cols) - set(white_cols))
oth_cols = [col for col in all_vap_cols if col != "VAP20" and col != "GEOID20"]

In [ ]:
race_mapping = {"BLACK": black_cols, 
                "ASIAN": asian_cols, 
                "HISP": hisp_cols, 
                "WHITE": white_cols, 
                "OTH": oth_cols}

In [ ]:
for race in race_mapping.keys():
    texas_vap_p4[race] =  texas_vap_p4[race_mapping[race]].sum(axis=1)

In [ ]:
tot_sum = 0
for race in race_mapping["OTH"]:
    tot_sum += sum(texas_vap_p4[race])
assert tot_sum == sum(texas_vap_p4["OTH"])

Here's a quick note to say that the BISG paper from JN and Bhushan necessarily use Pr(b|r), or given a block what is the likelihood of someone of that race living in that specific block. 
I believe the right calculation to do is the one below, but a second set of eyes on that wouldn't hurt!

I did this calculation in a Texas-specific way, but in the paper it's done on a national level (?) I think, so also double checking that would be helpful!!

In [ ]:
for race in race_mapping.keys():
    texas_vap_p4[f"{race}_prob"] = texas_vap_p4[race]/sum(texas_vap_p4[race])

In [ ]:
for race in race_mapping.keys():
    texas_vap_p4[f"{race}_share"] = texas_vap_p4[race]/texas_vap_p4["VAP20"]

In [ ]:
assert math.isclose(sum(texas_vap_p4["HISP_prob"]), 1, abs_tol = .001)

In [ ]:
race_prob_df = texas_vap_p4[["GEOID20", "BLACK_prob", "HISP_prob", "ASIAN_prob", "WHITE_prob", "OTH_prob", "BLACK_share", "HISP_share", "ASIAN_share", "WHITE_share", "OTH_share"]]
race_prob_df = race_prob_df.fillna(0)


In [ ]:
race_prob_df.to_csv("TX-block-race-prob.csv", index=False)

Here's a quick check. I'll do further comparisons against the data.mggg texas block shp

In [ ]:
all_vap_blocks = len(texas_vap_p4[texas_vap_p4.VAP20 > 0])
print(all_vap_blocks)
for col in race_prob_df.columns: 
    if "prob" in col: 
        col_name = col.split("_")[0]
        print(f"Share of populated blocks with {col_name} people: {len(race_prob_df[race_prob_df[col] > 0])/all_vap_blocks}")

In [ ]:
tx_county = gpd.read_file("http://data.mggg.org.s3-website.us-east-2.amazonaws.com/census-2020/tx/tx_county.zip")

In [ ]:
tx_county["EIOTH20"] = tx_county.VAP20 - tx_county.APBVAP20 - tx_county.HVAP20 - tx_county.ASIANVAP20

In [ ]:
tx_county_dict = {"BLACK": "APBVAP20", 
                  "HISP": "HVAP20", 
                  "ASIAN": "ASIANVAP20",
                  "OTH": "EIOTH20"}


In [ ]:
for k in tx_county_dict.keys():
    print(f"Group: {k}, Evaltools: {sum(texas_vap_p4[race_mapping[k]].sum(axis=1))/sum(texas_vap_p4.VAP20)}, data.mggg: {sum(tx_county[tx_county_dict[k]])/sum(tx_county.VAP20)}")

Another 1 time grab!

In this section I'm getting all of the surnames included in the census API (there are 162,253) which covers 90% of the people whose surnames were included at the time.

Open to any testing suggestions, as well. This is just not a place where we really have ground truths to check against.

In [ ]:
base_url = 'https://api.census.gov/data/2010/surname?get=NAME,'
groups = {"PCTWHITE":"WHITE", 
          "PCTBLACK": "BLACK", 
          "PCTHISPANIC": "HISP", 
          "PCTAPI": "ASIAN", 
          "PCTAIAN": "AMIN", 
          "PCT2PRACE": "2MORE"}

   
for group in sorted(list(groups.keys()))[:-1]:
        base_url += f"{group},"
base_url += f"{sorted(list(groups.keys()))[-1]}"
base_url += "&RANK=1:1000000"

req = requests.get(base_url).json()

pred_df = pd.DataFrame(req[1:], columns=req[0]).rename(columns=groups).drop(columns=["RANK"])
pred_df = pred_df.replace({'(S)':0})
pred_df["OTH"] = pred_df["AMIN"].astype(float) + pred_df["2MORE"].astype(float)
pred_df = pred_df.drop(columns=["AMIN", "2MORE"])
pred_df.to_csv("All-Surname-probs.csv", index=False)

In [ ]:
pred_df = pd.read_csv("All-Surname-probs.csv")

In [ ]:
for col in pred_df.columns[1:]:
    pred_df[col] = pred_df[col]/100

In [ ]:
pred_df.to_csv("All-Surname-probs.csv", index=False)

Prior to this, I'd already truncated the headers file to only include rows for Anderson county. 
This is for testing purposes only. I saved this to a tab separated value file, called `Anderson-Headers.tab`

This version of anderson is from a cleaned file created by Max called `voterfile-2017` meaning this only includes elections that happened 2017 and before. (This isn't particularly relevant until later).

In this section, I'm creating `anderson_mini`, a dataframe only including columns that will be relevant for testing. 

The header dataframe does have blocks, but the block ids included in that dataframe are not GEOIDs, so I'm also doing some of my own pre-processing to get map their block-ids to GEOID20s. This is how to read that
* I'm pulling the Texas county shape from data.mggg
* I'm creating a dictionary (saved in the first run), of county names to county fips
* Reconstructing the GEOID20 column from the other included info
    * Recall Block GEOID20 = (State FIPS) + (County FIPS) + (Tract ID) + (Block)

In [ ]:
anderson = pd.read_csv("ANDERSON_Header.csv", dtype={"Residence_Addresses_CensusTract":str, "Residence_Addresses_CensusBlock":str}).drop(columns=["Unnamed: 0"])

In [ ]:
anderson.head()

In [ ]:
print("Total voters in Anderson:", len(anderson))
print("Total voters in unidentified tracts in Anderson:", len(anderson[anderson["Residence_Addresses_CensusTract"].isna()]))
print("Total voters in unidentified blocks in Anderson:", len(anderson[anderson["Residence_Addresses_CensusBlock"].isna()]))

For my testing purposes, I'm dropping nan values.

In [ ]:
anderson = anderson.dropna(subset=['Residence_Addresses_CensusBlock'])

In [ ]:
tx_county = json.load(open("TX-county-name-fips.json"))

In [ ]:
anderson["COUNTYFP20"] = len(anderson) * [tx_county["Anderson"]]

In [ ]:
anderson["STATEFP20"] = len(anderson) * ['48']

In [ ]:
anderson["GEOID20"] = anderson["STATEFP20"].astype(str) + anderson["COUNTYFP20"].astype(str).str.zfill(3) + anderson["Residence_Addresses_CensusTract"] + anderson["Residence_Addresses_CensusBlock"]

In [ ]:
anderson_mini = anderson[['LALVOTERID', 'Voters_LastName', 'GEOID20']]

Including these TX specific checks, will need help debugging what I'm seeing. 

Essentially, I'm reconstructing GEOIDs as above, it does seem to create 1 weird NaN geoid (even tho I fillna), but there are a number of GEOIDs in the header file that aren't in the tx_block shp

The GEOIDs coming from the header file *look* valid, they just don't exist :/

Also putting this here, but reading in the tx_block shp is sooo slow. I'd maybe do it AFTER the BISG section if you're reviewing this. 

In [ ]:
tx_block = gpd.read_file("http://data.mggg.org.s3-website.us-east-2.amazonaws.com/census-2020/tx/tx_block.zip")

In [ ]:
max(tx_block[tx_block.COUNTYFP20 == "001"].GEOID20.unique())

In [ ]:
max(anderson_mini.GEOID20.unique())

In [ ]:
print(f"Anderson: {len(anderson_mini.GEOID20.unique())}, TX Block: {len(tx_block[tx_block.COUNTYFP20 == '001'].GEOID20.unique())}")

In [ ]:
len(list(set(anderson_mini.GEOID20.unique()) - set(tx_block[tx_block.COUNTYFP20 == "001"].GEOID20.unique())))

Ok, cool!

(I'm typing in real time sorry)
 
Now I have the race_prob_df and the surname_prob_df (and the anderson df). This is everything I need to do BISG!

Quickly running into issues that voters are mapped to a block with 0 pop. Either need to re-get GEOIDs from lat-long for the voters or figure out cases like this. 

In the case where either the name is not included in the surname prediction csv (census only included names with at least 5 occurrences) OR the denominator is 0, meaning that there were no people in the block, I give an equal weight to all possibilities. 

Now, up until this point, I've been keeping white and other separate for qa purposes, however, when EI is run, these will be combined. Looking for feedback about whether I should combine them before this step OR at this step give BLACK = 0.25, HISP = 0.25, ASIAN = 0.25, WHITE = 0.125, and OTH = 0.125 for the purposes of recombining. 

In [ ]:
pred_df = pd.read_csv("All-Surname-probs.csv")
race_prob_df = pd.read_csv("TX-block-race-prob.csv", dtype={"GEOID20":str})

In [ ]:
race_prob_df_test = race_prob_df[race_prob_df.BLACK_prob > 0.5]

In [ ]:
race_prob_df_test

In [48]:
voter_pred_df = pd.DataFrame(columns=["LALVOTERID", "LASTNAME", "GEOID20", "BLACK_prob", "HISP_prob", "ASIAN_prob", "WHITE_prob", "OTH_prob"])

# OK making bigger text here for something worth noting
There are 1211 unique GEOIDs found for Anderson. 
Of those only 841 are represented in the race_prob_df GEOIDs. 
This ends up with ~13000 out of ~25000 voters being represented. So HALF of the voters could be lost due to random GEOIDs. 

For now, I'll just fill those with 25% chance of being any of the racial groups

In [25]:
len(anderson.GEOID20.unique())

1211

In [41]:
for geoid in anderson.GEOID20.unique():
    assert len(geoid) == 15

In [26]:
len(anderson.GEOID20[anderson.GEOID20.isin(race_prob_df.GEOID20)].unique())

841

In [27]:
1211-841

370

# And now...BISG

I'm sure there's a smarter way to do this so it isn't as slow considering we'll have to do all counties. 

You can see the assertion statement included to ensure that the sum of final probabilities for each voter is close to 1. 
The reason for the is close instead of 1 is because I've seen some .9999.... or 1.0000.... values which I think still means the probability
is valid, and the point of error is just floating point stuff.

This took 40 minutes to run on Anderson county (25,000 voters-ish), so take a look at this, but I saved the outputs and read them in below.



In [32]:
andrews = pd.read_csv("Andrews-header16.csv", dtype={"GEOID20":str})

In [51]:
race_prob_df.GEOID20 = race_prob_df.GEOID20.astype(str)

In [56]:
%%time
for ix, row in tqdm.tqdm(andrews.iterrows()):
    voter_id = row["LALVOTERID"]
    name = row["Voters_LastName"]
    block_prob = race_prob_df[race_prob_df.GEOID20 == row.GEOID20]
    
    if name.upper() in list(pred_df.NAME):
        surname_prob = pred_df[["BLACK", "HISP", "ASIAN", "WHITE", "OTH"]][pred_df.NAME == name.upper()]
        voter_dict = {}
        for race in surname_prob.columns:
            voter_dict[race] = (float(surname_prob[race].values[0]) * float(block_prob[f"{race}_prob"].values[0]))
        denom = sum(voter_dict.values())
        if denom > 0: 
            voter_dict = {k: v/denom for k, v in voter_dict.items()}
            new_row = [voter_id, name, row.GEOID20, voter_dict["BLACK"], voter_dict["HISP"], voter_dict["ASIAN"], voter_dict["WHITE"], voter_dict["OTH"]]
        else: 
            print("denominator not greater than 0")
            new_row = [voter_id, name, row.GEOID20, block_prob.BLACK_share, block_prob.HISP_share, block_prob.ASIAN_share, block_prob.WHITE_share, block_prob.OTH_share] 
    else: 
        if name.upper() not in list(pred_df.NAME): 
                print(f"NAME: {name}")
        else: 
            print("GEOID20")
        new_row = [voter_id, name, row.GEOID20, block_prob.BLACK_share, block_prob.HISP_share, block_prob.ASIAN_share, block_prob.WHITE_share, block_prob.OTH_share] 
    assert math.isclose(sum(new_row[3:]), 1, abs_tol = .01)
    voter_pred_df.loc[ix] = new_row

20it [00:01, 13.21it/s]

GEOID20
GEOID20
GEOID20


24it [00:01, 14.52it/s]

GEOID20
NAME: Avitia Fernandez


30it [00:02, 13.77it/s]

...

GEOID20
GEOID20
NAME: Elcewicz


334it [00:26, 12.46it/s]

GEOID20


340it [00:26, 12.61it/s]


KeyboardInterrupt: 

In [29]:
voter_pred_df = pd.read_csv("Anderson-voter-predictions.csv")

In [54]:
voter_pred_df

,LALVOTERID,LASTNAME,GEOID20,BLACK_prob,HISP_prob,ASIAN_prob,WHITE_prob,OTH_prob
0,LALTX494641725,Hathcox,480039504001027,0.0,1.000000,0.0,0.000000,0.0
1,LALTX4462161,Trevino,480039504001049,0.0,0.978053,0.0,0.021947,0.0
2,LALTX1673620,Trevino,480039504001049,0.0,0.978053,0.0,0.021947,0.0
3,LALTX1053644,Trevino,480039504001049,0.0,0.978053,0.0,0.021947,0.0
4,LALTX1050354,Bartley,480039504001029,0.0,0.084043,0.0,0.915957,0.0
...,...,...,...,...,...,...,...,...
135,LALTX1032167,Ham,480039504001008,0.2,0.200000,0.2,0.200000,0.2
136,LALTX1032133,Ham,480039504001008,0.2,0.200000,0.2,0.200000,0.2
137,LALTX512731576,Perry,480039504001008,0.2,0.200000,0.2,0.200000,0.2
138,LALTX513086922,Sutton,480039504001008,0.2,0.200000,0.2,0.200000,0.2


In [31]:
print(len(voter_pred_df))

21030


In [32]:
print(len(voter_pred_df[voter_pred_df.BLACK_prob > 0]))
print(len(voter_pred_df[voter_pred_df.ASIAN_prob > 0]))
print(len(voter_pred_df[voter_pred_df.OTH_prob > 0]))
print(len(voter_pred_df[voter_pred_df.HISP_prob > 0]))

15027
10918
18986
18639


Ok, this is both a test and the beginning of post-processing. 

With every voter now BISG-ed, we can re-merge to the anderson dataframe and aggregate by GEOID. 

I'm writing this after running the cells, and these numbers are somewhat believable, except the Asian probability predicts way more Asian voters than seem to be in Anderson county. 

Wikipedia supports that there's a small number of Asian voters in Anderson, so...?

In [33]:
voter_pred_agg = anderson.merge(voter_pred_df, on = "LALVOTERID")

In [34]:
voter_pred_agg = voter_pred_agg[["GEOID20", "BLACK_prob", "HISP_prob", "ASIAN_prob", "OTH_prob"]]

In [35]:
voter_pred_agg = voter_pred_agg.groupby("GEOID20").sum().reset_index()

In [36]:
for col in list(voter_pred_agg.columns[1:]):
    print(f"{col}: {sum(voter_pred_agg[col])}")

BLACK_prob: 3546.607179353943
HISP_prob: 4545.687854497082
ASIAN_prob: 2482.4867511668795
OTH_prob: 10455.21821498209


In [37]:
county = gpd.read_file("http://data.mggg.org.s3-website.us-east-2.amazonaws.com/census-2020/tx/tx_county.zip")

In [38]:
county["EIOVAP20"] = county["VAP20"] - county["APBVAP20"] - county["HVAP20"] - county["ASIANVAP20"]

In [39]:
groups = ["APBVAP20", "HVAP20", "ASIANVAP20", "EIOVAP20"]
for group in groups: 
    print(f"{group}: {sum(county[group][county.NAME20 == 'Anderson'])}")

APBVAP20: 10146
HVAP20: 8350
ASIANVAP20: 317
EIOVAP20: 28568


# In theory the post processing is easy from here: 
    * We'll have BISG predictions on all counties for all election years. 
    * Concatenate each output to one large dataframe (only with the columns for GEOID20 and each probability)
    * Separately I'll make a mapping from every election date they have to the elections in our data set. Then for each unique voter, we'll figure out which elections they participated in, and sum the total votes for each election. 
     *  For each election date, there will be new pop columns that map to a speciifc data, and the pop data to be used in EI will specifically be the population numbers for each group on that day. i.e. BVAP201103 (sum of Black Voters for the November 3, 2020 election)
    * Aggregate from blocks to VTDs
    * Merge with current Texas.csv being used for EI!
    
    * And then there will be some EI-side changes before this is run-able